# LangWatch Scenario for Agent Testing

So far we used plain pytest for testing and also implemented a judge for that.

It's good, but there are also specialized tools for agent testing.

One of them is [LangWatch Scenario](https://github.com/langwatch/scenario). You can see their [demo here](https://www.youtube.com/watch?v=OHg02uRg5kE).

In this lesson I want to show you a simple case of using Scenario. If you like it, you can explore it more. I also find the implementation interesting - there are things to learn from reading the code.

It uses agents to test agents, which provides valuable insights into how these testing frameworks work internally.



## Installation and Setup

Install Scenario:

In [ ]:
!uv add --dev langwatch-scenario

Create a new test file: `_test_agent_scenario.py` (`tests/_test_agent_scenario.py`)

I put an underscore in front so it doesn't get picked up by default pytest discovery mechanisms.

This allows us to run these tests separately when we want to experiment with Scenario without affecting our regular test suite.

## Creating the Agent Adapter

The code in this lesson is based on this example: [examples/lovable_clone/lovable_agent.py](https://github.com/langwatch/scenario/blob/main/python/examples/lovable_clone/lovable_agent.py). It also uses Pydantic AI, so I decided to adapt it.

Since we use Pydantic AI, we need to create a wrapper:

In [ ]:
import scenario

from pydantic_ai.models.openai import OpenAIChatModel

from main import run_agent

class SearchAgentAdapter(scenario.AgentAdapter):

    async def call(self, input: scenario.AgentInput) -> scenario.AgentReturnTypes:
        user_prompt = input.last_new_user_message_str()
        result = await run_agent(user_prompt)
        new_messages = result.new_messages()
        return await self.convert_to_openai_format(new_messages)

    async def convert_to_openai_format(self, messages):
        openai_model = OpenAIChatModel("any")
        new_messages_openai_format = []
        for openai_message in await openai_model._map_messages(messages):
            new_messages_openai_format.append(openai_message)

        return new_messages_openai_format

This adapter bridges between our Pydantic AI agent and Scenario's expected interface.

The call method extracts the user's message and runs our agent. The `convert_to_openai_format` method transforms Pydantic AI's message format back to OpenAI's format, which Scenario expects for analysis.

## Defining a Test Scenario

Now we define a scenario:

In [ ]:
import pytest

@pytest.mark.asyncio
async def test_agent_code():
    result = await scenario.run(
        name="Evidentily Documentation",
        description="""
            The agent is tasked with asking questions about 'LLM as a Judge' evaluation.
            Send the first message to ask the question and then follow up
            with another question to understand the topic better. 
        """,
        agents=[
            SearchAgentAdapter(),
            scenario.UserSimulatorAgent(),
            scenario.JudgeAgent(
                criteria=[
                    "agent makes 3 search calls",
                    "the references are relevant to the topic",
                    "each section has references",
                    "the article contains properly formatted python code examples"
                ],
            ),
        ],
        max_turns=2,
        set_id="python-example",
    )

    assert result.success

Here the important parts are:

description guides the UserSimulatorAgent on how to interact with our search agent. It acts as a testing scenario specification.

JudgeAgent analyzes the entire interaction and checks if all criteria are satisfied. It's an LLM-based evaluator that can understand complex requirements.

JudgeCriteria.criteria are the specific requirements we want to verify. The JudgeAgent evaluates whether our agent meets these criteria.

UserSimulatorAgent pretends to be a user and uses the description to interact with our agent in a realistic way.

Like previously, we use LLM as a Judge. It can evaluate nuanced behaviors that would be difficult to check with traditional assertions.

## Running the Test

Let's run it:

In [ ]:
!uv run pytest tests/_test_agent_scenario.py::test_agent_code -s

We see a nice output:


```text
Total Scenarios: 1
Passed: 1
Failed: 0
Success Rate: 100.0%

1. Evidentily Documentation - PASSED
   Reasoning: The agent successfully made three search calls, gathered relevant information about LLM as a judge evaluation, and provided references for each section. Additionally, the content includes properly formatted Python code examples related to the evaluation process.
   Passed Criteria: 4/4
```

## When to Use Scenario vs Traditional Tests

Use Scenario when:

- You need complex multi-turn conversations
- You want to test realistic user interactions
- You need sophisticated evaluation criteria that are hard to code
- You want to simulate different user behaviors

Use traditional tests when:

- You need simpler unit tests
- You need to test specific functions or components
- You want fine-grained control over test execution

## Code from Video...

`tests/_test_agent_scenario.py`

In [ ]:
import pytest
import scenario
from pydantic_ai.models.openai import OpenAIChatModel
import main

scenario.configure(default_model="openai/gpt-4o-mini")

class SearchAgentAdapter(scenario.AgentAdapter):

    async def call(self, input: scenario.AgentInput) -> scenario.AgentReturnTypes:
        user_prompt = input.last_new_user_message_str()
        result = await main.run_agent(user_prompt)
        new_messages = result.new_messages()
        return await self.convert_to_openai_format(new_messages)

    async def convert_to_openai_format(self, messages):
        openai_model = OpenAIChatModel("any")
        new_messages_openai_format = []
        for openai_message in await openai_model._map_messages(messages):
            new_messages_openai_format.append(openai_message)

        return new_messages_openai_format
    
@pytest.mark.asyncio
async def test_agent_code():

    user_prompt = "How do I implement LLM as a Judge eval?"

    result = await scenario.run(
        name="Evidently Search Agent Code Test",
        description="""
            User asks for help with implementing LLM as a Judge evaluation in Evidently
        """,
        agents=[
            SearchAgentAdapter(),
            scenario.UserSimulatorAgent(),
            scenario.JudgeAgent(
                criteria=[
                    "Provides accurate and relevant code examples",
                    "Explains code implementation clearly",
                    "Contains at least one python code block in the article",
                    "Contains references"
                ],
            ),
        ],
        max_turns=2,
        set_id="python-example",
    )

    assert result.success

In [ ]:
!uv run pytest tests/_test_agent_scenario.py::test_agent_code